#**Sistema de Aprendizaje Profundo basado en Detección de Objetos para Evaluar Cursogramas Automáticamente**


---


Sistema software que lleva a cabo la evaluación de un Cursograma (imagen versión Alumno) comparándolo contra un Cursograma de referencia (imagen versión Docente), mediante la aplicación de un modelo ResNet previamente entrenado para detección la ubicación de los símbolos en ambas imágenes.

0) Preparar ambiente e instalar paquetes:

In [ ]:
#@title Forzar la utilización de TF versión 1 para compatibilidad 
# nota se debe indicar la versión 1 de TF para compatibilidad del código
%tensorflow_version 1.x
import tensorflow as tf
print("Usando TF ", tf.__version__)

In [ ]:
#@title Baja e instala los parquetes de 'Object Detection' de Tensor Flow a utilizar en el disco temporal de Colab (demora un ratito)
!pip install tf_slim

%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
model_dir = 'training/'

!python object_detection/builders/model_builder_test.py

os.environ['PYTHONPATH'] += ':/content/models/research/object_detection:/content/models/research/slim/object_detection'


1) Cargar librerías:

In [ ]:
#@title Cargar las librerías necesarias

import os
import os.path
import sys
import numpy as np
import csv

from IPython.display import Image, display
from PIL import Image as ImPIL

import tensorflow as tf
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont

from difflib import SequenceMatcher

from google.colab import files
import time

print ("Librerías cargadas")

2) Montar el Drive:

In [ ]:
#@title Montar el drive y definir las configuración de las carpetas a usar

# monta Google Drive:
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# configuración de directorios local en Google Drive
drive_path = '/content/gdrive/My Drive/GEMIS/objDetectionCursogramas'
data_dir_path = drive_path + '/Cursogramas'
model_dir_path = drive_path + '/TF_model'
evaluar_dir_path = data_dir_path + '/Para-Evaluar'

print("Configuración de archivos definida")

3) Cargar el modelo entrenado:

In [ ]:
#@title Carga el modelo de object detection entrenado

# path donde está el modelo exportado
ModelObjDetEntrenado = model_dir_path + '/frozen_inference_graph.pb'

# archivo con lista de etiquetas para mostrar 
labelMapFile = model_dir_path + '/label_map.pbtxt'

# se debe ubicar en el directorio correspondiente
%cd /content/models/research/object_detection

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

# This is needed to display the images.
%matplotlib inline

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
 
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(ModelObjDetEntrenado, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(labelMapFile)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=90, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

# función auxiliar para ejecutar el modelo
def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict


print("Modelo objDetector-Carteles cargado: [", ModelObjDetEntrenado, "], [", labelMapFile, "] ")


In [ ]:
#@title Carga la configuración de los símbolos del Cursograma

# carga la información de un archivo CSV
config_simbolos_file = model_dir_path +'/simbolos_config.csv'
with open(config_simbolos_file, mode='r') as csvfile:
    lineasCSV = list(csv.reader(csvfile))

# procesa el archivo CSV
auxList = []
for l in lineasCSV:
  # si no se debe ignorar el simbolo
  if l[0][0] != "#":     
      # carga la configuración del símbolo (aunque se ignore en la comparación)
      # campo clave: nombre - valores: [descripción, claseAgrupa, ignora?, cambiarAreaY?, cambiarAreaX?] 
      auxList.append( ( l[0], [l[1], int(l[2]), (l[3]=="1"), (l[4]=='1'), (l[5]=='1')] ) )

# genera dicccionario de la configuración de símbolos
config_simbolos = dict(auxList)

print(">Configuración de símbolos: ")
print("  cargada de [", config_simbolos_file, "] ")
print("  valores: nombre + [descrip, [descripción, claseAgrupa, ignora?, cambiarAreaY?, cambiarAreaX?] ] ")
print("  ", config_simbolos)

4) Determina los parámetros para lleva a cabo la evaluación:

In [ ]:
#@title Indique las imágenes a procesar: { run: "auto" }

# la imagen de referencia para evaluar
imagen_docente = 'curso_docente_01.png' #@param [ "curso_docente_01.png", "curso_docente_02.png", curso_docente_03.png", "curso_docente_04.png", "curso_docente_05.png" ] {allow-input: true}

# define las imágenes de alumnos a considerar
cargarAutomaticamenteImagenesAlumnos = True  #@param {type:"boolean"}
lista_imagen_alumno = []
if cargarAutomaticamenteImagenesAlumnos:

  # lista auxiliar con las imágenes de alumnos a procesar 
  lista_imagen_alumno = [ fn for fn in os.listdir( evaluar_dir_path ) if fn.endswith('.png') or fn.endswith('.jpg') ]
  ## if (fn[0:7] == imagen_docente[0:7]) 
  lista_imagen_alumno = sorted(lista_imagen_alumno)  
else:

  # la imagen a ser evaluada
  imagen_alumno = 'curso_alumno_15.png' #@param {type:"string"}
  lista_imagen_alumno.append( imagen_alumno )

print("Archivos a procesar: ")
print("   Docente: ", imagen_docente)
print("   Alumno: ", lista_imagen_alumno)

In [ ]:
#@title Defina parámetros a utilizar: { run: "auto" }

# define minima probabilidad a usar
minimaProbabilidadObjectosDetectados = 95 
minProbObjDet = minimaProbabilidadObjectosDetectados / 100.

# define parámetro Intersection over Union (IoU) 
## si calc_IoU(r1, r2) ≥ coefIoU, se considera que se detectó el objecto correctamente, es Verdadero Positivo (VP)
## si calc_IoU(r1, r2) < coefIoU, se considera que se detectó el objecto con error, es Falso Positivo (FP)
## -> valor recomendado por defecto: 0,5 
## pero se usa menos para mejorar los resultados
coefIoU = 0.3 

# indica los rangos de puntaje para asignar la 'Nota'
# debe quedar ordenado en orden decreciente
rango_notas = [ [100, "EXCELENTE"],
               [95, "MUY BIEN"],
               [85, "BIEN+"], 
               [75, "BIEN"], 
               [65, "BIEN-"], 
               [55, "REGULAR"],
               [50, "REGULAR-"],
               [0, "MAL"] ]
              
# tipos de letras para usar
im_font = ImageFont.truetype('/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf', 14)
im_fontPequenia = ImageFont.truetype('/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf', 10)

# define si muestra detalle o no
muestraDetalleDebug = False  #@param {type:"boolean"}
muestraDetalleObjDetectadosEnImagen = False  #@param {type:"boolean"}
muestraDetalleMetricasComparacion = False  #@param {type:"boolean"}
muestraDetalleComparacionEnImagen = "Todas" #@param [ "Solo con Error", "Todas"]

# define los colores a utilizar
# orden: OK, Faltan, Sobra, Diferentes, textoEncabezado
muestraDiferentesColoresComparacionEnImagen = True  #@param {type:"boolean"}
if muestraDiferentesColoresComparacionEnImagen:
  lista_colores_comparacion = [ (0,255,0), (255,0,0), (150,0,150), (255,75,0), (111,0,0) ]
else:
  lista_colores_comparacion = [ (0,255,0), (255,0,0), (255,0,0), (255,0,0), (111,0,0)]

# define si se muestra la posición que le correspondeería 
# al objeto Docente en la imagen Alumno (para probar solamente)
muestraPosicionObjetoDocenteEnImagenAlumno = False  #@param {type:"boolean"}

# define que las imágenes Alumno evaluadas que se muestran se guardan en drive y/o bajan localemente automaticamente 
guardarEnDriveImagenesAlumnoEvaluadas = True  #@param {type:"boolean"}
bajarAutomaticamenteImagenesAlumnoEvaluadas = False  #@param {type:"boolean"}

# define la carpeta donde se almacenan los diagramas evaluados
if guardarEnDriveImagenesAlumnoEvaluadas or bajarAutomaticamenteImagenesAlumnoEvaluadas: 
  evaluar_dir_path_resultados = evaluar_dir_path + "/Resultados/"
  if not os.path.isdir(evaluar_dir_path_resultados):
      os.makedirs(evaluar_dir_path_resultados)

print("Parámetros definidos")

5) Lleva a cabo la evaluación:

In [ ]:
#@title Define las funciones auxiliares a usar

# función auxiliar para procesar una imagen ya cargada con el modelo 
# y devuelve la lista de objetos detectados que superan una probabilidad
# con la lista de áreas X generadas
def obtener_objetos_imagen(imageCargada, minProbObjDet=0.8, muestraDetalleObjDetectadosEnImagen=False):

    if muestraDetalleDebug:
      print("   obtener_objetos_imagen -- Parámetros: minProbObjDet (", minProbObjDet, ")  - muestraDetalleObjDetectadosEnImagen (", muestraDetalleObjDetectadosEnImagen, ") ")

    # define el tamaño de la imagen
    imCargada_ancho, imCargada_alto = imageCargada.size

    # prepara la imagen cargada
    image_np = load_image_into_numpy_array(imageCargada)
    image_np_expanded = np.expand_dims(image_np, axis=0)

    # ejecuta el modelo
    output_dict = run_inference_for_single_image(image_np, detection_graph)

    if muestraDetalleObjDetectadosEnImagen: 

        # Marca los objetos detectados en la imagen
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
            category_index,
            instance_masks=output_dict.get('detection_masks'),
            use_normalized_coordinates=True,
            line_thickness=8)

        # muestra la imagen con los objetos detectados
        display( ImPIL.fromarray(image_np, 'RGB') )

    # iniciala la lista de objetos detectados
    listObjsDetModelo = []

    # procesa los objetos detectados
    for detClass, detBox, detScore in zip(  output_dict['detection_classes'], output_dict['detection_boxes'], output_dict['detection_scores'] ):

        # si el objeto detectado tiene un puntaje superior o igual al mínimo
        if detScore >= minProbObjDet:

              class_name = category_index[detClass]['name']

              # como las coordenadas están normalizadas las debe convertir 
              # teniendo en cuenta el tamaño de la imagen
              # además notar que vienen datas en otro orden
              # - detBox = (ini alto, ini ancho, fin alto, fin ancho)
              # - nuevoRangoIn = (ini ancho x1, ini alto y1, fin ancho x2, fin alto y2)    
              nuevoRangoIm = [ round(detBox[1] * imCargada_ancho,0), 
                              round(detBox[0] * imCargada_alto,0),
                              round(detBox[3] * imCargada_ancho,0),
                              round(detBox[2] * imCargada_alto,0) ]

                # agrega a lista de objetos detectados por el modelo
              # indicando (clase, [x1, y1, x2, y2] )
              listObjsDetModelo.append( ( class_name, nuevoRangoIm ) )          

    return listObjsDetModelo

# función auxiliar para calcular el Intersection over Union (IoU) 
# versión obtenida de https://towardsdatascience.com/evaluating-performance-of-an-object-detection-model-137a349c517b
def calc_IoU( gt_bbox, pred_bbox):
    '''
    This function takes the predicted bounding box and ground truth bounding box and 
    return the IoU ratio
    '''
    x_topleft_gt, y_topleft_gt, x_bottomright_gt, y_bottomright_gt = gt_bbox
    x_topleft_p, y_topleft_p, x_bottomright_p, y_bottomright_p = pred_bbox
    
    if (x_topleft_gt > x_bottomright_gt) or (y_topleft_gt> y_bottomright_gt):
        raise AssertionError("Ground Truth Bounding Box is not correct")
    if (x_topleft_p > x_bottomright_p) or (y_topleft_p> y_bottomright_p):
        raise AssertionError("Predicted Bounding Box is not correct",x_topleft_p, x_bottomright_p,y_topleft_p,y_bottomright_gt)
         
    #if the GT bbox and predcited BBox do not overlap then iou=0
    if(x_bottomright_gt< x_topleft_p):
        # If bottom right of x-coordinate  GT  bbox is less than or above the top left of x coordinate of  the predicted BBox
        
        return 0.0
    if(y_bottomright_gt< y_topleft_p):  
      # If bottom right of y-coordinate  GT  bbox is less than or above the top left of y coordinate of  the predicted BBox
        
        return 0.0
    if(x_topleft_gt> x_bottomright_p): 
      # If bottom right of x-coordinate  GT  bbox is greater than or below the bottom right  of x coordinate of  the predcited BBox
        
        return 0.0
    if(y_topleft_gt> y_bottomright_p): 
      # If bottom right of y-coordinate  GT  bbox is greater than or below the bottom right  of y coordinate of  the predcited BBox
        
        return 0.0
    
    GT_bbox_area = (x_bottomright_gt -  x_topleft_gt + 1) * (  y_bottomright_gt -y_topleft_gt + 1)
    Pred_bbox_area =(x_bottomright_p - x_topleft_p + 1 ) * ( y_bottomright_p -y_topleft_p + 1)
    
    x_top_left =np.max([x_topleft_gt, x_topleft_p])
    y_top_left = np.max([y_topleft_gt, y_topleft_p])
    x_bottom_right = np.min([x_bottomright_gt, x_bottomright_p])
    y_bottom_right = np.min([y_bottomright_gt, y_bottomright_p])
    
    intersection_area = (x_bottom_right- x_top_left + 1) * (y_bottom_right-y_top_left  + 1)
    
    union_area = (GT_bbox_area + Pred_bbox_area - intersection_area)
   
    if union_area > 0.0:
        return intersection_area/union_area
    else:
        return 0.0


# función auxiliar para ordenar los objetos detectados y asignar las áreas correspondientes
# nota: la lista no necesita estar ordenada porque la acomoda acá
def organizar_objDet(listObjsDetModelo, imAncho, imAlto):
    
    if muestraDetalleDebug:       
      print("\n> Función organizar_objDet: ")

    # pre-inicializa las áreas de eje X (columnas)
    tamanioColAreaX = 100
    listAreasX = []

    # pre-inicializa considerando los objetos que producen cambio área X 
    # (por ejemplo "Translado-Info-Horizontal")
    for obj in listObjsDetModelo:
        if obj[0] in config_simbolos:
          # si es un símbolo que produce cambio de área X
          if config_simbolos[obj[0]][4]:
              minAreaX = obj[1][0] - 1
              maxAreaX = obj[1][2] + 1
              # define 3 áreas X (anterior, cambioArea, posterior)
              listAreasX.append( [minAreaX-tamanioColAreaX, minAreaX-1, 0] )
              listAreasX.append( [minAreaX, maxAreaX, 0] )
              listAreasX.append( [maxAreaX+1, maxAreaX+tamanioColAreaX, 0] )

    # ordena las áreas X de izquierda a derecha
    listAreasX = sorted(listAreasX, key=lambda col:col[0])

    if muestraDetalleDebug:  
      print("   - Pre-inicialización de Áreas X: ", listAreasX)
    
    # controla que no haya ninguna área X que se solape con otras
    # si se encuentra, se elimina por considerarla redundante
    i = 0
    while i < len(listAreasX):
        if (i > 0) and (listAreasX[i][0] < listAreasX[i-1][1]):
          if (i < len(listAreasX)-1) and (listAreasX[i][1] > listAreasX[i+1][0]):
                if muestraDetalleDebug: 
                  print("        se elimina área X ", listAreasX[i], " por superposición con contiguas.")
                listAreasX.pop( i )
                i = i - 1
        i = i + 1

    # ordena los objetos de arriba hacia abajo, y de izquierda a derecha
    # - notar que a los objetos que generan cambios de áreas Y los adelanta un poco 
    # para que incluyan otros que estén en la misma línea horizontal 
    # y que pueden estar ubicados un poco más arriba
    listObjsDetModelo = sorted(listObjsDetModelo, key=lambda obj:( (obj[1][1] - (15 if config_simbolos[obj[0]][3] else 0)) * 1000000 + obj[1][0] ) )

    # define la cantidad y tamaño de áreas eje Y para particionar una imagen 
    cantAreasYImagen = 2
    areaAlto = imAlto / cantAreasYImagen
    
    # contador de cambios de area Y    
    cambiosAreaY = 0

    # lista auxiliar para devolver resultado
    listObjetosOrg = []

    # procesa los objetos para organizar por área
    for obj in listObjsDetModelo:

        # obtiene la configuración del símbolo
        if obj[0] in config_simbolos:
            descSimbolo = config_simbolos[obj[0]][0]
            claseAgrupa = config_simbolos[obj[0]][1]
            consideraCompara = not( config_simbolos[obj[0]][2] )
            cambiarAreaY = config_simbolos[obj[0]][3]
            cambiarAreaX = config_simbolos[obj[0]][4]
        else:
            descSimbolo = obj[0]
            claseAgrupa = -1
            consideraCompara = False            
            cambiarAreaY = False
            cambiarAreaX = False

        # si determina cambio de área Y 
        # (por ejemplo "Temporalidad" o "Decisión")
        if cambiarAreaY:
            # fuerza el cambio de area
            cambiosAreaY = cambiosAreaY + 100
            for arX in listAreasX:
                arX[2] = arX[2] + 100
            # define el area Y
            areaY = cambiosAreaY
        else:
            # asigna area Y de acuerdo a posición y1 de la figura
            areaY = int(obj[1][1] // areaAlto) + cambiosAreaY

        # si la figura es muy ancha para ser casi toda la figura
        if abs((obj[1][2]-obj[1][0])-imAncho)<100:
            # define el area sin area X
            areaObj = [-1, areaY]
            # determina la posición relativa del objeto de acuerdo a su área Y solamente
            posOrden = areaY
        else:
            # asigna area X de acuerdo al centro de la figura 
            # teniendo en cuenta la posición x1 & x2 de la figura
            areaX = -99
            i = 0
            cenFigura = ( obj[1][0] + obj[1][2] ) // 2
            while (areaX == -99) and (i<len(listAreasX)):
              # si centro de la figura se encuentra dentro de las coordenadas definidas para el área X
              if listAreasX[i][0] <= cenFigura and cenFigura <= listAreasX[i][1]: 
                    areaX = i
              i = i + 1

            # si no tiene definida área X asigna una nueva de acuerdo a x1 & x2
            if (areaX == -99):             
                minAreaX = cenFigura - ( tamanioColAreaX // 2 )
                maxAreaX = minAreaX + tamanioColAreaX
                # agrega nueva área X
                listAreasX.append( [minAreaX, maxAreaX, cambiosAreaY] )
                areaX = len(listAreasX)-1

            # define area consolidada
            areaObj = [areaX, areaY]
            # aumenta contador de objetos dentro del área X
            listAreasX[areaX][2] = listAreasX[areaX][2] + 1
            # determina la posición relativa del objeto de acuerdo a su áreas (X, Y)
            posOrden = areaX*10000 + listAreasX[areaX][2]

        # registra el objeto con la siguiente información
        # (posorden, [idAreaX, idAreaY], descripción, [x1, y1, x2, y2], [consideraCompara, claseAgrupa, cambioAreaY, cambioAreaX])
        listObjetosOrg.append( (posOrden, areaObj, descSimbolo, obj[1], [consideraCompara, claseAgrupa, cambiarAreaY, cambiarAreaX]) )
        
    # vuelve a ordenar de arriba hacia abajo, y de izquierda a derecha
    listObjetosOrg = sorted(listObjetosOrg, key=lambda obj:obj[1][1]*1000000+obj[0])

    # determina cual es el área X principal 
    # (considerando cuál es la que tiene el primer símbolo)
    ppal_areaX = listObjetosOrg[0][1][0]

    if muestraDetalleDebug:
        print("   - Áreas Y: alto: ", areaAlto, " / cant: ", cantAreasYImagen, " \ cambiosAreaY: ", cambiosAreaY)
        print("   - Áreas X (ID ppal", ppal_areaX, "): ", listAreasX)        
        print("   - Lista ordenada de objetos y organizada por áreas: ")
        for objDet_org in listObjetosOrg:
            print("            ", objDet_org)     
        print("\n")

    return listObjetosOrg, listAreasX, ppal_areaX


# función auxiliar para mostrar boxes en una imagen dada
def draw_box(draw, rangeObj, lineWidth, lineColor):  

    # dibuja el rectángulo
    draw.line([(rangeObj[0], rangeObj[1]), 
                        (rangeObj[0], rangeObj[3]), 
                        (rangeObj[2], rangeObj[3]), 
                        (rangeObj[2], rangeObj[1]), 
                        (rangeObj[0], rangeObj[1])], 
                      width=lineWidth, fill=lineColor)

# función auxiliar para mostrar boxes de una lista objetos en una imagen dada
def draw_boxes_listObj(draw, listObj, lineWidth, lineColor):
  
    for obj in listObj:
        draw_box(draw, obj[2], lineWidth, lineColor)

# función auxiliar para mostrar línea de unión entre 2 boxes
def draw_union_boxes_listObj(draw, posObj1, posObj2, lineWidth, lineColor):

  # determina posición destino
  p2x = posObj2[0]
  p2y = posObj2[3]

  # determina posición origen
  if abs(posObj1[0]-p2x) < abs(posObj1[2]-p2x):
      p1x = posObj1[0]
  else:
      p1x = posObj1[2]
  p1y = posObj1[3]

  # actualiza posición x de origen si hace falta
  if abs(posObj2[0]-p1x) > abs(posObj2[2]-p1x):
      p2x = posObj2[2]

  # dibuja la línea de unión
  draw.line([(p1x, p1y), (p2x, p2y) ], width=lineWidth, fill=lineColor)

# función auxiliar para mostrar un texto asociado a un box
def draw_text(draw, rangeObj, texto, font, color):

    # acomoda la posición x para que no se pase del tamaño de la imagen
    posX = rangeObj[2]+5
    posY = rangeObj[1]
    (fwidth, fheight), (offset_x, offset_y) = font.font.getsize(texto) 
    im_ancho = draw.im.size[0]
    if (posX+fwidth) > im_ancho:
          posX = rangeObj[2] - fwidth - 5
          posY = posY + 5

    # muestra el texto
    draw.text((posX, posY), texto, fill=color, font=font)


# función auxiliar para incrementar imagen para agregar cabecera en blanco al inicio 
# y usarlo para mostrar un texto asociado a un box
def draw_image_new_header(img, comp_draw, texto):

  # calcula el tamaño de imagen que se debería incrementar
  ascent, descent = im_font.getmetrics() 
  tamanioIncrementarTop = 5
  texto_posY = [ tamanioIncrementarTop ]
  for txt in texto:     
    tamanioIncrementarTop = tamanioIncrementarTop + (ascent + descent) 
    texto_posY.append( tamanioIncrementarTop )  
  # agrega sangría en blanco arriba
  tamanioIncrementarTop = tamanioIncrementarTop + (ascent + descent) * 2

  # define el texto para agregar en el pie
  textFooter = "Referencia <" + imagen_docente + ">"
  tamanioIncrementarBottom = (ascent + descent)

  # aumenta el tamaño de la imagen (arriba y abajo)
  imgN, imgN_draw = cambiarTamanioImagen(img, tamanioIncrementarTop, tamanioIncrementarBottom)
  im_width, im_height = imgN.size

  # escribe el nuevo texto en la cabecera
  for txt, posY in zip(texto, texto_posY):
    draw_text(imgN_draw, [5, posY, 5, posY], txt, im_font, lista_colores_comparacion[4])
  
  # escribe el nuevo texto en el pie
  draw_text(imgN_draw, [(im_width*0.80), (im_height-tamanioIncrementarBottom), (im_width*0.80), (im_height-tamanioIncrementarBottom)], 
            textFooter, im_fontPequenia, lista_colores_comparacion[4])

  return imgN, imgN_draw

# función auxiliar para calcular y devolver la nota correspondiente a una imagen alumno
def calcularNotaAlumno():

  total = (metricasImag[posVP] + metricasImag[posFP_r] + metricasImag[posFP_i] + metricasImag[posFN])
  if total>0:     
      puntaje = round(100*metricasImag[posVP]/total, 0)     
      i = 0
      while ( i < len(rango_notas) ) and ( rango_notas[i][0] > puntaje ): i = i + 1
      if muestraDetalleDebug:  
          print("\n Puntaje: ", puntaje, " -> Nota: ", rango_notas[i] )
      
      return "   Nota: " + rango_notas[i][1] 
  else:
      if muestraDetalleDebug:  
          print("\n No se puede calcular puntaje ")
      return ""

# función auxiliar para marcar en la imagen las observaciones correspondientes 
# y devuelve el texto para la cabecera
def generarObservaciones():

    global resObservaciones
    # si no hay observaciones devuelve vacío
    if (resObservaciones == None) or (len(resObservaciones) ==  0):
      return ["   No hay observaciones."]

    # arma el texto para el encabezado
    textoObsCabecera = []  
    textoObsCabecera.append("   Observaciones: " )

    # ordena las observaciones según posición en imagen
    # de arriba hacia abajo, y de izquierda a derecha     
    resObservaciones = sorted(resObservaciones, key=lambda obj:obj[0][1]*1000+obj[0][2])

    # procesa las observaciones [posImagen, colorImagen, tipoProblema, texto]
    # mostrando texto con la referencia en la imagen 
    #  y armando el texto para la cabecera
    refId = 0
    for obs in resObservaciones:
        refId = refId + 1
        refText =  '(' + str(refId) + ')'
        draw_text(comp_draw, obs[0], refText, im_font, obs[1]) 
        textoObsCabecera.append( '    ' + obs[2] + ' ' + refText + ':  ' + obs[3] )

    return textoObsCabecera

# función auxiliar para calcular y mostrar las métricas correspondientes a una imagen alumno
def mostrarMetricas(imagen_alumno):

    print("\n= Matriz de Confusión para la Imagen del Alumno ", imagen_alumno, ":")        
    print("                  ALUMNO ")
    print(" DOCENTE   :       +          -   ")
    print("    +      :     %3d        %3d  " % (metricasImag[posVP], metricasImag[posFN]) )
    print("    -      :     %3d        %3d  " % (metricasImag[posFP_i]+metricasImag[posFP_r], 0.0) )

    # Cálculo de la Exactitud
    total = (metricasImag[posVP] + metricasImag[posFP_r] + metricasImag[posFP_i] + metricasImag[posFN])
    if total>0:
        print("= Exactitud: ", round(100*metricasImag[posVP]/total, 3))

    # Cálculo de la Precisión
    total = (metricasImag[posVP] + metricasImag[posFP_i] + metricasImag[posFP_r])
    if total>0:
        print("= Precisión: ",  round(100*metricasImag[posVP]/total,3))

    # Cálculo de la Recuperación
    total = (metricasImag[posVP] + metricasImag[posFN])
    if total>0:
        print("= Recuperación: ", round(100*metricasImag[posVP]/total,3))       


# función auxiliar para cambiar el tamaño de una imagen incrementando su tamaño (agregando espacio arriba y/o abajo y/o derecha)
def cambiarTamanioImagen(img, incrementarCabecera=0, incrementarPie=0, incremetarAncho=0):

  # si no tiene nada para incrementar, devuelve la misma imagen
  if (incrementarCabecera==0) and (incrementarPie==0) and (incremetarAncho==0):
      return img 
  # incrementa el tamaño de la imagen 
  im_ancho, im_alto = img.size   
  imgN = img.crop( (0, (incrementarCabecera*(-1)), (im_ancho+incremetarAncho), (im_alto+incrementarPie)))  
  imgN_draw = ImageDraw.Draw(imgN)
  # pinta de blanco el nuevo espacio agregado en la imagen
  if incrementarCabecera > 0:
      imgN_draw.rectangle( (0, 0, im_ancho, incrementarCabecera), fill="white")
  if incrementarPie > 0:
      imgN_draw.rectangle( (0, (incrementarCabecera+im_alto), im_ancho, (incrementarCabecera+im_alto+incrementarPie)), fill="white")
  if incremetarAncho > 0:
    imgN_draw.rectangle( (im_ancho, 0, (im_ancho+incremetarAncho), (incrementarCabecera+im_alto+incrementarPie)), fill="white")
  # devuelve la nueva imagen
  return imgN, imgN_draw

# función auxiliar que verificar si un box estaría dentro de una imagen, y sino la incrementa
def verificarObjPosEnImagen(rangeObj):
    
    global comp_image_pil
    global comp_draw
    
    if (comp_image_pil == None):
        return False

    res = False    
    im_ancho, im_alto = comp_image_pil.size
    # ajusta la altura
    if (im_alto < rangeObj[3]):        
        incrImAlto = rangeObj[3] - im_alto + 5
    else:
        incrImAlto = 0
    # ajusta el ancho    
    if (im_ancho < rangeObj[2]):        
        incrImAncho = rangeObj[2] - im_ancho + 5
    else:
        incrImAncho = 0
    # si se debe incrementar
    if (incrImAlto + incrImAncho) > 0:  
        comp_image_pil, comp_draw = cambiarTamanioImagen(comp_image_pil, 0, incrImAlto, incrImAncho)
        res = True
        if muestraDetalleDebug:
          print("            * verificarBoxEnImagen: se incrementa imagen: incrImAlto ", incrImAlto, "& incrImAncho ", incrImAncho)        

    return res

# función auxiliar para ajustar la posición de rangeObj 
# teniendo en cuenta la última posición Y, así como también las áreas X de las imágenes
# devuelve la posición ajustada
def determinaPosicionCorrectaDocenAl(rangeObj, ultPosY=0, idAreaXDoc=-1, areasX_doc=[], areasX_al=[], difIDAreasX=0, defaultPosAreasX=0):
  
    # determina coordenadas del objeto que falta
    auxBox = [ rangeObj[0], rangeObj[1], rangeObj[2], rangeObj[3] ]
       
    # si la lista de areas X están definida, ajusta la posición X del objeto
    if (idAreaXDoc >= 0) and (len(areasX_doc) > idAreaXDoc):
        # determina el ID que corresponde al área X de Alumno    
        idAreaXAl = idAreaXDoc + difIDAreasX
        # si el ID área X de Alumno es válida (puede ser positiva o negativa)
        if  (len(areasX_al) > abs(idAreaXAl)):
            difPosAreasX = areasX_al[idAreaXAl][0] - areasX_doc[idAreaXDoc][0]    
        else:
            # toma la diferencia por defeecto entre áreas X
            difPosAreasX = defaultPosAreasX
        # ajusta las posiciones X conisderando la configuración de las áreas X
        auxBox[0] = auxBox[0] + difPosAreasX
        auxBox[2] = auxBox[2] + difPosAreasX  

        # si la posiciones X quedan negativas las cambia hacia la derecha
        # toma la posición X de la última área X + constante
        if (auxBox[2] < 0):
          difPosAreasX = areasX_al[-1][0] + 150           
          auxBox[0] = difPosAreasX - (rangeObj[2] - rangeObj[0])//2
          auxBox[2] = auxBox[0] + rangeObj[2]- rangeObj[0]                     
                   
        if muestraDetalleDebug:
          print("            * determinaPosicionCorrectaDocenAl params: ultPosY: ", ultPosY, "& idAreaXDoc: ", idAreaXDoc, "& idAreaXAl: ", idAreaXAl, "& difPosAreasX: ", difPosAreasX)
    else:
        if muestraDetalleDebug:
          print("            * determinaPosicionCorrectaDocenAl params: ultPosY: ", ultPosY, "& idAreaXDoc: ", idAreaXDoc)

    # si el recuadro quedaría ubicado arriba que otro ya marcado 
    # se cambia su posición Y para que quede más abajo
    #if auxBox[3] < ultPosY:
    auxBox[3] = ultPosY + auxBox[3] - auxBox[1] + 15
    auxBox[1] = ultPosY + 15

    if muestraDetalleDebug:
        print("            * determinaPosicionCorrectaDocenAl: ajusta ", rangeObj, " --> ", auxBox)

    return auxBox

# función auxiliar para ajustar la posición de rangeObj para que sea más pequeña
# devuelve las posiciones ajustadas
def reducirTamanioBox(rangeObj):

    # determina coordenadas del objeto que falta
    auxBox = [ rangeObj[0], rangeObj[1], rangeObj[2], rangeObj[3] ]
    
    # se reduce el tamaño del box por 1/4
    ajusteRedX = (auxBox[2] - auxBox[0]) // 4
    ajusteRedY = (auxBox[3] - auxBox[1]) // 4
    auxBox[0] = auxBox[0] + ajusteRedX
    auxBox[1] = auxBox[1] + ajusteRedY
    auxBox[2] = auxBox[2] - ajusteRedX
    auxBox[3] = auxBox[3] - ajusteRedY 

    return auxBox


# función auxiliar para mostrar los resultados de la comparación 
# cuando es igual en alumno y docente
# devuelve la última posición Y del objeto
def procesar_compara_igual(objDet):    

    # actualiza las métricas
    metricasImag[posVP] = metricasImag[posVP] + 1      
    # dibuja recuadro de símbolo
    draw_box(comp_draw, objDet[3], 2, lista_colores_comparacion[0]) 
    # devuelve la última posición Y del objeto
    return objDet[3][3]

# función auxiliar para mostrar los resultados de la comparación cuando falta en alumno
def procesar_compara_falta(objDet_doc, newRangeObj_Doc): 

    # reduce el tamaño del box y agranda la imagen si es necesario para que entre
    box = reducirTamanioBox(newRangeObj_Doc)
    verificarObjPosEnImagen(box)
    # dibuja recuadro de símbolo faltante
    draw_box(comp_draw, box, 4, lista_colores_comparacion[1]) 
    # actualiza las métricas
    metricasImag[posFN] = metricasImag[posFN] + 1       
    #  agrega observación
    resObservaciones.append( [box, lista_colores_comparacion[1], '-', 'Falta incluir "' + objDet_doc[2] + '".'] )
    # devuelve la última posición Y del objeto
    return box[3]

# función auxiliar para mostrar los resultados de la comparación cuando sobra en alumno
def procesar_compara_sobra(objDet_al):

    # dibuja recuadro de símbolo
    draw_box(comp_draw, objDet_al[3], 4, lista_colores_comparacion[2]) 
    # actualiza las métricas
    metricasImag[posFP_i] = metricasImag[posFP_i] + 1
    #  agrega observación
    resObservaciones.append( [objDet_al[3], lista_colores_comparacion[2], '+', 'No corresponde incluir "' + objDet_al[2] + '".'] )    
    # devuelve la última posición Y del objeto
    return objDet_al[3][3]

# función auxiliar para mostrar los resultados de la comparación cuando hay alguna diferencia de clase 
def procesar_compara_distClase(objDetDoc, objDetAl):

    # determina texto específico a utilizar en observaciones
    msg = 'Se indica "' + objDetAl[2] + '" pero debe ser "' + objDetDoc[2] + '".'
    # devuelve la última posición Y del objeto
    return procesar_compara_dist( objDetAl, msg, lista_colores_comparacion[3] )

# función auxiliar para mostrar los resultados de la comparación cuando hay alguna diferencia de clase 
def procesar_compara_distArea(objDetDoc, newRangeObj_Doc, objDetAl): 

    # reduce el tamaño del box y agranda la imagen si es necesario para que entre
    box_doc = reducirTamanioBox(newRangeObj_Doc)
    verificarObjPosEnImagen(box_doc)
    # indica cuadro con la posición correcta
    draw_union_boxes_listObj(comp_draw, objDetAl[3], box_doc, 1, lista_colores_comparacion[3])
    draw_box(comp_draw, box_doc, 2, lista_colores_comparacion[3]) 
    # determina texto específico a utilizar en observaciones
    msg = 'Se indica "' + objDetAl[2] + '" en una ubicación incorrecta.'  
    # marca en la imagen  
    procesar_compara_dist( objDetAl, msg, lista_colores_comparacion[3] )
    # devuelve la última posición Y del objeto
    return box_doc[3]

# función auxiliar para mostrar los resultados de la comparación cuando hay alguna diferencia de clase o área
def procesar_compara_dist(objDetAl, msgTexto, tColor):

    # dibuja recuadro de símbolo           
    draw_box(comp_draw, objDetAl[3], 4, tColor) 
    # actualiza las métricas
    metricasImag[posFP_r] = metricasImag[posFP_r] + 1    
    #  agrega observación
    resObservaciones.append( [objDetAl[3], tColor, '~', msgTexto] )        
    # devuelve la última posición Y del objeto
    return objDetAl[3][3]

print("Funciones auxiliares definidas")   

In [ ]:
#@title Procesa la imagen del Docente para generar la referencia
print("> procesando diagrama docente ", imagen_docente, ": ")

 # carga la imagen a procesar
fn_imagen_docente = evaluar_dir_path + '/Docente/' + imagen_docente
imageCargada_doc = ImPIL.open(fn_imagen_docente) 

# Convierte la imagen a escala de grises y luego a RGB 
# (para sacarle los colores que tuviera previamente y dejarlo con 3 canales de profundidad)
imageCargada_doc = imageCargada_doc.convert('L')
imageCargada_doc = imageCargada_doc.convert('RGB')

im_width_doc, im_height_doc = imageCargada_doc.size 

# procesa la imagen
list_objDet_doc = obtener_objetos_imagen(imageCargada_doc, minProbObjDet, muestraDetalleObjDetectadosEnImagen)

# muestra los resultados
if muestraDetalleDebug:
  print("\n    ", len(list_objDet_doc), " simbolos detectados en Docente: ")
  for objDet in list_objDet_doc:
      print("   ", objDet)     

# Si no se muestran los objetos dtectados, muestra la imagen de referencia
if not(muestraDetalleObjDetectadosEnImagen):
  display( imageCargada_doc )

# ordena y organiza por área a los objetos detetados
# para la imagen de docente 
list_objDet_doc_org, lista_areasX_doc_org, ppal_areaX_doc_org = organizar_objDet(list_objDet_doc, im_width_doc, im_height_doc)

# determine información para comparar
compara_doc = [ (obj[2], obj[1][0], obj[1][1]) for obj in list_objDet_doc_org]   


In [ ]:
#@title Procesa las imágenes de Alumno para realizar la Evaluación  

# chequea que tenga la información de la imagen Docente
if compara_doc==None or list_objDet_doc_org==None:
  print("No se posee información de la imagen Docente de referencia!")
  exit 

# procesa las imágenes Alumno
for imagen_alumno in lista_imagen_alumno:

  print("\n\n> procesando diagrama alumno ", imagen_alumno, ": ")

  if muestraDetalleDebug:
    tiempoInicioProcesarAlumno = time.time()

  # carga la imagen a procesar
  fn_imagen_alumno = evaluar_dir_path + '/' + imagen_alumno
  imageCargada_al = ImPIL.open(fn_imagen_alumno) 

  # Convierte la imagen a escala de grises y luego a RGB 
  # (para sacarle los colores que tuviera previamente y dejarlo con 3 caneles de profundidad)
  imageCargada_al = imageCargada_al.convert('L')
  imageCargada_al = imageCargada_al.convert('RGB')

  im_width, im_height = imageCargada_al.size 

  # procesa la imagen
  list_objDet_al = obtener_objetos_imagen(imageCargada_al, minProbObjDet, muestraDetalleObjDetectadosEnImagen)

  if muestraDetalleDebug:
    print(" \n - Procesar imagen para obtener objetos lleva ", (time.time() - tiempoInicioProcesarAlumno), "segundos ")

  # muestra los resultados
  if muestraDetalleDebug:
    print("\n    ", len(list_objDet_al), " símbolos detectados en Alumno:")
    for objDet in list_objDet_al:
        print("   ", objDet)

  # ordena y organiza por área a los objetos detetados
  # para la imagen de docente 
  # list_objDet_al_org => (orden, [idAreaX, idAreaY], descripción, [x1, y1, x2, y2], [consideraCompara, claseAgrupa, cambioAreaY, cambioAreaX])
  # lista_areasX_al_org => idAreaX -> (minX, maxX)
  # ppal_areaX_al_org {ID principal área X}
  list_objDet_al_org, lista_areasX_al_org, ppal_areaX_al_org = organizar_objDet(list_objDet_al, im_width, im_height)

  # determine información para comparar
  difIDAreasX = ppal_areaX_al_org - ppal_areaX_doc_org
  difPosPpalAreasX = lista_areasX_al_org[ppal_areaX_al_org][0] - lista_areasX_doc_org[ppal_areaX_doc_org][0]  
  compara_al = [ (obj[2], (obj[1][0] - difIDAreasX), obj[1][1]) for obj in list_objDet_al_org] 
   
  if muestraDetalleDebug:
      print(" \n - Organizar objetos lleva ", (time.time() - tiempoInicioProcesarAlumno), "segundos ")
      print("\n> Realiza  la comparación: ")
      print("  difIDAreasX: ", difIDAreasX, " & defDifPosAreasX: ", difPosPpalAreasX)
      print("  Clases Docente: ", compara_doc)
      print("  Clases Alumno: ", compara_al)

  # realiza la comparación 
  # usando  una solución propuesta 
  # en [https://stackoverflow.com/questions/3462143/get-difference-between-two-lists]
  dif = SequenceMatcher(None, 
                        compara_doc, 
                        compara_al)

  # vector auxiliar para mostrar las metricas
  metricasImag = [ 0, 0, 0, 0]
  posVP = 0
  posFP_i = 1
  posFP_r = 2
  posFN = 3
  resObservaciones = []

  # imagen auxiliar para mostrar resultados comparación
  ##comp_image_pil = copy.deepcopy(imageCargada_al.convert("RGB"))
  comp_image_pil = imageCargada_al.convert("RGB")
  comp_draw = ImageDraw.Draw(comp_image_pil)

  # procesa los resultados de la comparación
  if muestraDetalleDebug:  
    print("\n> Procesa la comparación: ")

  # en la primera vuelta sólo marca los que son correctos (iguales) 
  # dejando los otros para un segundo procesamiento 
  ultPosListIgual = -1
  problIdsObj_doc = []
  problIdsObj_al = []
  for tag, i, j, k, l in dif.get_opcodes():

    if muestraDetalleDebug:    
        print("  ", ("==" if tag == 'equal' else ("--" if  tag == 'delete' else ("++" if tag == 'insert' else "~~" ) )  ) )
        print("    Doc[", i, ":",  j, "]=", compara_doc[i:j] )
        print("    Al[ ", k, ":", l, "]=", compara_al[k:l] )
    
    if tag == 'equal': 
        # objetos iguales en Ambos
        for pos in range(k, l):
          # si no se ignora
          if list_objDet_al_org[pos][4][0]:
            procesar_compara_igual( list_objDet_al_org[pos] )
          # actualiza última posición Y
          ultPosListIgual = pos      
    else:
          # agrega ids de objetos de docente con problemas que no se ignoran
          for pos in range(i, j):
            if list_objDet_doc_org[pos][4][0]:
              problIdsObj_doc.append( [pos, ultPosListIgual] )
          # agrega ids de objetos de alumno con problemas que no se ignoran
          for pos in range(k, l):
            if list_objDet_al_org[pos][4][0]:
                problIdsObj_al.append( [pos, ultPosListIgual] )

  # realiza la segunda vuelta considerando los que tuvieron problemas
  # porque difieren, son sobrantes o faltantes
  if muestraDetalleDebug:  
    print("\n> Continuá con los que generaron problemas en la comparación: ")
    print("   problIdsObj_doc: ", problIdsObj_doc)
    print("   problIdsObj_al: ", problIdsObj_al)  
  # variables auxiliares
  ultPosY = 0
  ultPosYAreaX = []
  for i in range(len(lista_areasX_doc_org)):
    ultPosYAreaX.append( 0 )
  auxCambioAreaYCompara = 0
  # procesa la lista de docente primero tratando de asociar por misma clase o área
  for itemProbl_doc in problIdsObj_doc:
      # carga el símbolo docente a procesar
      pos_doc, ultPosListIgual_doc = itemProbl_doc
      objDoc = list_objDet_doc_org[pos_doc]
      if muestraDetalleDebug: 
          print("    # procesando objDoc ", objDoc, ":")
      # actualiza último pos Y 
      if (ultPosListIgual_doc >= 0): 
        if (list_objDet_al_org[ultPosListIgual_doc][1][0] < 0) or (objDoc[1][0] < 0):
            # considera posición inferior y último pos Y (si no es mayor, se queda con ese)
            if (list_objDet_al_org[ultPosListIgual_doc][3][3] > ultPosY):
                ultPosY = list_objDet_al_org[ultPosListIgual_doc][3][3]
        elif ((list_objDet_al_org[ultPosListIgual_doc][1][0]-difIDAreasX) == objDoc[1][0]):
            # considera posición inferior por estar en el mismo área X
            if (list_objDet_al_org[ultPosListIgual_doc][3][3] > ultPosYAreaX[objDoc[1][0]]):
                ultPosY = list_objDet_al_org[ultPosListIgual_doc][3][3]
            else:
                ultPosY = ultPosYAreaX[objDoc[1][0]] 
        else:
            # considera posición superior por estar en distinta área X
            if (list_objDet_al_org[ultPosListIgual_doc][3][1] > ultPosYAreaX[objDoc[1][0]]):
                  ultPosY = list_objDet_al_org[ultPosListIgual_doc][3][1]
            else:
                  ultPosY = ultPosYAreaX[objDoc[1][0]]
      # determina la posición que le correspondería al objeto docente en la imagen alumno      
      newPosObjDoc = determinaPosicionCorrectaDocenAl(rangeObj=objDoc[3], ultPosY=ultPosY, idAreaXDoc=objDoc[1][0], 
                                              areasX_doc=lista_areasX_doc_org, areasX_al=lista_areasX_al_org, 
                                              difIDAreasX=difIDAreasX, defaultPosAreasX=difPosPpalAreasX)  
      if muestraPosicionObjetoDocenteEnImagenAlumno:
          # muestra la ubicación que correspondería en la imagen Alumno
          draw_box(comp_draw, newPosObjDoc, 1, (100,100,100))           
      # evalua los simbolos alumnos con problemas
      noEncAlumno = True
      for itemProbl_al in problIdsObj_al:
          # carga el símbolo alumno a procesar
          pos_al, ultPosListIgual_al = itemProbl_al
          objAl = list_objDet_al_org[pos_al]
          # calcula IoU con respecto al objeto Alumno
          IoUnDocAl = calc_IoU(newPosObjDoc, objAl[3])          
          # evalúa                       
          if objAl[2] == objDoc[2]:               
              # si tiene igual clase ...
              if (IoUnDocAl >= coefIoU):
                  # ... y misma ubicación (considerando IoU entre objetos Doc y Al) 
                  if muestraDetalleDebug: 
                    print("       =  se rectifica con objAl ",  objAl)
                    print("             calc_IoU: ", IoUnDocAl, "& coefIoU: ", coefIoU, "& ultPosY: ", ultPosY, "& auxCambioAreaYCompara: ", auxCambioAreaYCompara)     
                  # actualiza imagen y última posición Y
                  ultPosY = procesar_compara_igual( objAl )
                  noEncAlumno = False
              else:
                  # ... y distinta ubicación
                  if (objAl[1][1] + auxCambioAreaYCompara) <= objDoc[1][1]:
                    # ... pero respetando el área X para que no se ademasiado diferente
                    if muestraDetalleDebug: 
                      print("       ~  se encuentra igual clase pero distinta área con objAl ",  objAl)
                      print("             calc_IoU: ", IoUnDocAl, "& coefIoU: ", coefIoU, "& ultPosY: ", ultPosY, "& auxCambioAreaYCompara: ", auxCambioAreaYCompara)   
                    # actualiza imagen y última posición Y
                    ultPosY = procesar_compara_distArea(objDoc, newPosObjDoc, objAl)              
                    noEncAlumno = False
          else:
              # si tiene distinta clase, pero son del mismo "claseAgrupa" ...
              if (objDoc[4][1] == objAl[4][1]):
                # compara la posición en la que debería estar el objeto Docente con el objeto en Alumno
                if (IoUnDocAl >= coefIoU):
                    # ... y mismas misma ubicación (considerando IoU entre objetos Doc y Al) 
                        if muestraDetalleDebug: 
                          print("       ~  se encuentra distinta clase pero misma área: con objAl ",  objAl)
                          print("             calc_IoU: ", IoUnDocAl, "& coefIoU: ", coefIoU, "& ultPosY: ", ultPosY, "& auxCambioAreaYCompara: ", auxCambioAreaYCompara)  
                        # actualiza imagen y última posición Y
                        ultPosY = procesar_compara_distClase(objDoc, objAl) 
                        noEncAlumno = False        
          
          if not(noEncAlumno):      
            # actualiza la última pos Y del área X  
            ultPosYAreaX[objDoc[1][0]] = ultPosY
            # saca de la lista auxiliar de alumno porque ya se utilizó
            problIdsObj_al.remove( itemProbl_al )           
            # sale del loop del for porque no tiene sentido seguir evaluando
            break
        
      if noEncAlumno:
          # si no se encontró, quiere decir que falta en Alumno
          # registra en imagen
          if muestraDetalleDebug: 
              print("       -  no se encuentra doc: ", objDoc)
              print("             ultPosY: ", ultPosY, "& auxCambioAreaYCompara: ", auxCambioAreaYCompara) 
          # actualiza imagen y última posición Y
          ultPosY = procesar_compara_falta(objDoc, newPosObjDoc)        
          if objDoc[1][0] < 0:
              # si el área X es menor que cero, 
              # fuerza la actualización de todos los últimos pos Y
              for i in range(len(ultPosYAreaX)):
                ultPosYAreaX[i] = ultPosY
          else:
              # actualiza la última pos Y del área X
              ultPosYAreaX[objDoc[1][0]] = ultPosY
          # si es un objeto que produce cambio de area Y
          if objDoc[4][2]:
            auxCambioAreaYCompara = auxCambioAreaYCompara + 100  
            #  fuerza la actualización de los otros  últimos pos Y
            #  con la posición y1 del objeto
            for i in range(len(ultPosYAreaX)):
                if objDoc[1][0] != i:
                  ultPosYAreaX[i] = newPosObjDoc[1]               
    
  # luego de procesar todos los objetos docentes, 
  # todos los que queden en la  lista de alumnos se toman como sobrantes
  for posA, ultPosYA in problIdsObj_al:
      # carga el símbolo alumno a procesar
      objAl = list_objDet_al_org[posA]     
      # si no se ignora
      if objAl[4][0]:           
          # registra en la imagen 
          if muestraDetalleDebug: 
              print("       +  sobra al: ", objAl)
              print("             ultPosY: ", ultPosY, "& auxCambioAreaYCompara: ", auxCambioAreaYCompara)                   
          # actualiza imagen y última posición Y
          ultPosY =  procesar_compara_sobra( objAl )           
      # si es un objeto que produce cambio de area Y
      if objAl[4][2]:
          auxCambioAreaYCompara = auxCambioAreaYCompara - 100    

  if muestraDetalleDebug:  
    print(" \n - Comparar objetos lleva ", (time.time() - tiempoInicioProcesarAlumno), "segundos ")

  # genera el texto para el encabezado
  textoHeader = []
  textoHeader.append( "> Resultados de la Evaluación del Cursograma <" + imagen_alumno + ">:" )
  
  # Cálcula y devuelve la nota del cursograma
  textoHeader.append( calcularNotaAlumno() )

  # Genera y devuelve las observaciones
  textoHeader.extend( generarObservaciones() )
 
  print("\n")
  if (muestraDetalleComparacionEnImagen == "Todas") or (metricasImag[posFP] > 0) or (metricasImag[posFN] > 0):

          # agrega el texto de la cabecera a la imagen
          comp_image_pil, comp_draw = draw_image_new_header(comp_image_pil, comp_draw, textoHeader)

          # muestra la imagen con los resultados de la comparación
          #imMostrar = ImPIL.fromarray(np.array(comp_image_pil), 'RGB')
          display( comp_image_pil )

          # guarda y/o baja la imagen con las correcciones marcadas
          # por el evaluado automaticamente
          if guardarEnDriveImagenesAlumnoEvaluadas or bajarAutomaticamenteImagenesAlumnoEvaluadas:
            nomArchi = "Eval_curso_doc_" + imagen_docente[14:16] + "_al_" + imagen_alumno[13:15] + ".png" 
            comp_image_pil.save(evaluar_dir_path_resultados + nomArchi)
            if bajarAutomaticamenteImagenesAlumnoEvaluadas:
                try: 
                  files.download(evaluar_dir_path_resultados + nomArchi)
                  print('Imagen ' + nomArchi + ' generada, debería bajarse automaticamente como un archivo local...')
                except ValueError:
                  print('Error al intentar descargar el archivo ' + nomArchi)

  else:
          # muestra el texto de la cabecera          
          for txt in textoHeader:
              print( txt )
          print("\n")

  if muestraDetalleMetricasComparacion:        
      mostrarMetricas(imagen_alumno)

  if muestraDetalleDebug:
    print(" \n - Procesamiiento total  lleva ", (time.time() - tiempoInicioProcesarAlumno), "segundos ")
